
# Wikidata rule suggestions


This notebook presents some possible rules that could be extacted from Wikidata.

Note that Wikidata and DBpedia switch the position of the object and subject.
So **P40[Child](A, B)** in Wikidata means that A is the child of B, while in DBpedia **child(A, B)** means that A has child B.

---
### **Rule 1**: P26[Spouse](A, B) -> P26[Spouse](B, A)
"If A is the spouse of B, then B is a spouse of A."

---
### **Rule 2**: P25[Mother](A, B) -> P40[Child](B, A)
"If A is the mother of B, then B is the child of A"

---
### **Rule 3**: p22[Father](A, B) -> P40[Child](B, A)
"If A is the father of B, then B is a child of A."

---
### **Rule 4**: P185[doctoralStudent](A, B) -> P184[doctoralAdvisor](B, A)
"If A is the doctoral student of B, then B is the doctoral advisor of A."

---

### **Rule 5**: P800[notableWork](A, B) -> P170[creator](B, A)
"If A is the notable work of B, then B is the creator of A."

---
### **Rule 6**: P1196[mannerOfDeath](A, Q171558[accident]) -> P184[causeOfDeath](A, Q171558[accident])
"If the manner of A's death was an accident, then the cause of A's death was a accident."

---

In [17]:
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
from pykeen.datasets import Wikidata5M
import pykeen
import tensorflow as tf
import pandas as pd
import numpy as np
from signature_tools import subset_by_signature, subset_by_strict_signature, subset_by_frequency, most_frequent_objects, most_frequent_predicates, most_frequent_targets
from ampligraph.evaluation import train_test_split_no_unseen 
from ampligraph.latent_features import ComplEx
from ampligraph.latent_features import save_model
from ampligraph.evaluation import evaluate_performance
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score
from ampligraph.latent_features import save_model

In [3]:
data_unprocessed = Wikidata5M()
data_unprocessed.summarize()

You're trying to map triples with 156 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 156 from 5133 triples were filtered out
You're trying to map triples with 180 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 180 from 5163 triples were filtered out


Wikidata5M (create_inverse_triples=False)
Name        Entities    Relations      Triples
----------  ----------  -----------  ---------
Training    4594149     822           20614279
Testing     4594149     822               4977
Validation  4594149     822               4983
Total       -           -             20624239
Head    Relation    tail
------  ----------  --------
Q1      P1343       Q602358
Q1      P1419       Q1647152
Q1      P1552       Q11412
Q1      P2184       Q136407
Q1      P2670       Q18343



In [4]:
# extract all the data into a numpy array of triples
train_Wikidata = data_unprocessed.training.triples
test_Wikidata = data_unprocessed.testing.triples
validation_Wikidata = data_unprocessed.validation.triples
data_Wikidata = np.concatenate((train_Wikidata, test_Wikidata, validation_Wikidata))
data_Wikidata = data_Wikidata.astype('object') # used to have datatype '<U95' which was problematic for signature_tools functions

Reconstructing all label-based triples. This is expensive and rarely needed.
Reconstructing all label-based triples. This is expensive and rarely needed.
Reconstructing all label-based triples. This is expensive and rarely needed.


In [6]:
most_frequent_objects(data_Wikidata, n=100)

KeyboardInterrupt: 

In [7]:
most_frequent_predicates(data_Wikidata, n=100)

array([[3841627, 'P31'],
       [1375274, 'P17'],
       [1146848, 'P27'],
       [1101408, 'P106'],
       [919905, 'P54'],
       [919489, 'P131'],
       [856424, 'P19'],
       [838752, 'P735'],
       [514458, 'P161'],
       [485845, 'P641'],
       [439909, 'P69'],
       [429980, 'P47'],
       [404947, 'P421'],
       [374369, 'P105'],
       [370254, 'P136'],
       [348802, 'P171'],
       [279197, 'P20'],
       [261001, 'P495'],
       [238803, 'P1412'],
       [222716, 'P1344'],
       [183473, 'P175'],
       [182448, 'P166'],
       [181086, 'P413'],
       [154179, 'P264'],
       [149046, 'P155'],
       [141736, 'P156'],
       [136593, 'P364'],
       [136331, 'P361'],
       [126546, 'P102'],
       [108630, 'P150'],
       [108322, 'P734'],
       [99586, 'P57'],
       [96454, 'P463'],
       [95780, 'P279'],
       [94036, 'P407'],
       [82598, 'P159'],
       [80172, 'P108'],
       [76208, 'P39'],
       [75654, 'P3373'],
       [71660, 'P937'],
       [7047

In [ ]:
most_frequent_targets(data_Wikidata, n=10)

In [8]:
# P8810 = parent of the subject. Only use if neither father (P22) nor mother (P25) is applicable
#P24 = child
#P26 = spouse
# P25 mother of the subject. For stepmother, use "stepparent" (P3448)
#P1038 = relative
#P3373 = sibling
#P3448 = steparent
family_subset = subset_by_signature(data_Wikidata, [], ['P24', 'P22', 'P25', 'P26', 'P1038', 'P3373', 'P3448'], [])

In [9]:
family_subset.shape

(193358, 3)

In [11]:
X_train, X_test = train_test_split_no_unseen(family_subset, test_size=100) 

Our data is now split into train/test sets. If we need to further divide into a validation dataset we can just repeat using the same procedure on the test set (and adjusting the split percentages). 

In [12]:
print('Train set size: ', X_train.shape)
print('Test set size: ', X_test.shape)

Train set size:  (193258, 3)
Test set size:  (100, 3)


In [13]:
["Trondheim_Airport", "hasGender", "Brisbane_Airport"] in family_subset

False

---
# 3. Training a model 

AmpliGraph has implemented [several Knoweldge Graph Embedding models](https://docs.ampligraph.org/en/latest/ampligraph.latent_features.html#knowledge-graph-embedding-models) (TransE, ComplEx, DistMult, HolE), but to begin with we're just going to use the [ComplEx](https://docs.ampligraph.org/en/latest/generated/ampligraph.latent_features.ComplEx.html#ampligraph.latent_features.ComplEx) model (with  default values), so lets import that:

In [14]:
model = ComplEx(batches_count=100, 
                seed=0, 
                epochs=200, 
                k=150, 
                eta=5,
                optimizer='adam', 
                optimizer_params={'lr':1e-3},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

## Filtering negatives

AmpliGraph aims to follow scikit-learn's ease-of-use design philosophy and simplify everything down to **`fit`**, **`evaluate`**, and **`predict`** functions. 

However, there are some knowledge graph specific steps we must take to ensure our model can be trained and evaluated correctly. The first of these is defining the filter that will be used to ensure that no *negative* statements generated by the corruption procedure are actually positives. This is simply done by concatenating our train and test sets. Now when negative triples are generated by the corruption strategy, we can check that they aren't actually true statements.  


In [15]:
positives_filter = family_subset

## Fitting the model

Once you run the next cell the model will train. 

On a modern laptop this should take ~3 minutes (although your mileage may vary, especially if you've changed any of the hyper-parameters above).

In [18]:
tf.logging.set_verbosity(tf.logging.ERROR)

model.fit(X_train, early_stopping = False)

Average ComplEx Loss:   0.006580:   8%| | 16/200 [27:28<5:16:00, 103.05s/epoch]


KeyboardInterrupt: 

In [ ]:
save_model(model, './Wikidata_family_subset.pkl')

This will save the model in the ampligraph_tutorial directory as `best_model.pkl`.

.. we can then delete the model .. 

In [ ]:
#del model

.. and then restore it from disk! Ta-da! 

In [ ]:
#model = restore_model('./connectedTo_subset.pkl')

And let's just double check that the model we restored has been fit:

In [ ]:
if model.is_fitted:
    print('The model is fit!')
else:
    print('The model is not fit! Did you skip a step?')

## Running evaluation

In [ ]:
ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=positives_filter,   # Corruption strategy filter defined above 
                             use_default_protocol=True, # corrupt subj and obj separately while evaluating
                             verbose=True)

In [ ]:
mrr = mrr_score(ranks)
print("MRR: %.2f" % (mrr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))